In [271]:
import pandas as pd
import numpy as np

In [272]:
#importing data
housing_data = pd.read_csv('../data/price_index.csv')
interest_rate_data = pd.read_csv('../data/canadian_interest_rates.csv')
income_data = pd.read_csv('../data/income_data.csv')
population_data = pd.read_csv('../data/calgary_population.csv')
median_house_price_data = pd.read_csv('../data/median_assessed_value.csv')


In [273]:
median_house_price_data = pd.read_csv('../data/median_assessed_value.csv')
median_house_price_data.rename(columns={'ROLL_YEAR': 'REF_DATE'}, inplace=True)
median_house_price_data['REF_DATE'] = pd.to_datetime(median_house_price_data['REF_DATE'], format='%Y')
median_house_price_data = median_house_price_data.set_index('REF_DATE').resample('MS').ffill()
print(median_house_price_data.head(24))  # Shows first year of monthly data


            median_re_assessed_value
REF_DATE                            
2005-01-01                    206500
2005-02-01                    206500
2005-03-01                    206500
2005-04-01                    206500
2005-05-01                    206500
2005-06-01                    206500
2005-07-01                    206500
2005-08-01                    206500
2005-09-01                    206500
2005-10-01                    206500
2005-11-01                    206500
2005-12-01                    206500
2006-01-01                    221000
2006-02-01                    221000
2006-03-01                    221000
2006-04-01                    221000
2006-05-01                    221000
2006-06-01                    221000
2006-07-01                    221000
2006-08-01                    221000
2006-09-01                    221000
2006-10-01                    221000
2006-11-01                    221000
2006-12-01                    221000


In [274]:
# Take a quick look at the data
print(housing_data.head())
print(interest_rate_data.head())
print(income_data.head())
print(population_data.head())

  REF_DATE              GEO           DGUID New housing price indexes  \
0  1981-01           Canada  2016A000011124    Total (house and land)   
1  1981-01           Canada  2016A000011124                House only   
2  1981-01           Canada  2016A000011124                 Land only   
3  1981-01  Atlantic Region      2016A00011    Total (house and land)   
4  1981-01  Atlantic Region      2016A00011                House only   

                 UOM  UOM_ID SCALAR_FACTOR  SCALAR_ID      VECTOR  COORDINATE  \
0  Index, 201612=100     347         units          0  v111955442         1.1   
1  Index, 201612=100     347         units          0  v111955443         1.2   
2  Index, 201612=100     347         units          0  v111955444         1.3   
3  Index, 201612=100     347         units          0  v111955445         2.1   
4  Index, 201612=100     347         units          0  v111955446         2.2   

   VALUE STATUS  SYMBOL  TERMINATED  DECIMALS  
0   38.2    NaN     NaN   

In [275]:
#Viewing Columns Names
print(housing_data.columns)
print(interest_rate_data.columns)
print(income_data.columns)
print(population_data.columns)

Index(['REF_DATE', 'GEO', 'DGUID', 'New housing price indexes', 'UOM',
       'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE',
       'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'],
      dtype='object')
Index(['REF_DATE', 'GEO', 'DGUID', 'Rates', 'UOM', 'UOM_ID', 'SCALAR_FACTOR',
       'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL',
       'TERMINATED', 'DECIMALS'],
      dtype='object')
Index(['REF_DATE', 'GEO', 'DGUID', 'Age group', 'Sex', 'Income source',
       'Statistics', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR',
       'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS'],
      dtype='object')
Index(['date', ' Population', ' Annual Change'], dtype='object')


In [276]:
#Dropping other provinces
housing_data = housing_data[housing_data.GEO == 'Calgary, Alberta']
#Filter for "Total (house and land)" category
housing_data = housing_data[housing_data['New housing price indexes'] == 'Total (house and land)']


#Dropping unused columns, axis=1 means dropping rows, inplace=True changes the original dataframe
housing_data.drop(['DGUID', 'New housing price indexes', 'UOM',
       'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE',
       'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS', 'GEO'], axis=1, inplace=True)


interest_rate_data.drop(['DGUID', 'Rates', 'UOM', 'UOM_ID', 'SCALAR_FACTOR',
                         'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS', 
                         'SYMBOL', 'TERMINATED', 'DECIMALS', 'GEO'], axis=1, inplace=True)



#Filtering data by Median Income, Better than average as high earnings skew data
income_data = income_data[income_data['Statistics'] == 'Median income (excluding zeros)']

#Dropping Unused Columns
income_data.drop(['DGUID', 'Age group', 'Sex', 'Income source',
        'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR',
       'COORDINATE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS','Statistics', 'GEO'], axis=1, inplace=True)


In [277]:
#Converting ref_date to datetime format
housing_data.REF_DATE = pd.to_datetime(housing_data.REF_DATE)
interest_rate_data.REF_DATE = pd.to_datetime(interest_rate_data.REF_DATE)
population_data.date = pd.to_datetime(population_data.date)
# This will convert the year-based REF_DATE into a proper datetime format that Pandas recognizes as the start of the year 
income_data['REF_DATE'] = pd.to_datetime(income_data['REF_DATE'], format='%Y')


In [278]:
housing_data.set_index('REF_DATE', inplace=True)
interest_rate_data.set_index('REF_DATE', inplace=True)
income_data.set_index('REF_DATE', inplace=True)
population_data.set_index('date', inplace=True)

In [279]:
#Checking for NAN values
housing_data.isna().values.any()
interest_rate_data.isna().values.any()
income_data.isna().values.any()
population_data.isna().values.any()

True

In [280]:
#Checking for dupliactes
print(housing_data.duplicated())
print(interest_rate_data.duplicated())
print(income_data.duplicated())
print(population_data.duplicated())

REF_DATE
1981-01-01    False
1981-02-01    False
1981-03-01    False
1981-04-01    False
1981-05-01     True
              ...  
2024-04-01    False
2024-05-01    False
2024-06-01    False
2024-07-01    False
2024-08-01    False
Length: 524, dtype: bool
REF_DATE
1981-01-07    False
1981-01-14    False
1981-01-21    False
1981-01-28     True
1981-02-04    False
              ...  
2024-09-04     True
2024-09-11     True
2024-09-18     True
2024-09-25     True
2024-10-02     True
Length: 2283, dtype: bool
REF_DATE
1981-01-01    False
1982-01-01    False
1983-01-01    False
1984-01-01    False
1985-01-01    False
1986-01-01    False
1987-01-01    False
1988-01-01     True
1989-01-01    False
1990-01-01    False
1991-01-01    False
1992-01-01    False
1993-01-01    False
1994-01-01    False
1995-01-01    False
1996-01-01    False
1997-01-01    False
1998-01-01    False
1999-01-01    False
2000-01-01    False
2001-01-01    False
2002-01-01    False
2003-01-01    False
2004-01-01    False
20

In [281]:
# Resample to monthly frequency using forward-fill to fill the year for each month, 'MS' is month start frequency
population_monthly = population_data.resample('MS').ffill()
interest_rate_monthly = interest_rate_data.resample('MS').mean()
income_monthly = income_data.resample('MS').ffill()  


# Reset index to make 'date' a column again
population_monthly.reset_index(inplace=True)
interest_rate_monthly.reset_index(inplace=True)
income_monthly.reset_index(inplace=True)


# Rename 'date' to 'REF_DATE'
population_monthly.rename(columns={'date': 'REF_DATE'}, inplace=True)

# Ensure 'REF_DATE' is in monthly format
population_monthly['REF_DATE'] = pd.to_datetime(population_monthly['REF_DATE'])
population_monthly['REF_DATE'] = population_monthly['REF_DATE'].dt.to_period('M').dt.to_timestamp()

population_monthly.dropna(inplace=True)


# Filter population data to include only records from 1981 onwards, This is to match the other data sets
population_monthly = population_monthly[population_monthly['REF_DATE'] >= '1981-01-01']





In [282]:
# View the result
print(housing_data.head())
print(population_monthly.head())
print(interest_rate_monthly.head())
print(income_monthly.head())


            VALUE
REF_DATE         
1981-01-01   25.3
1981-02-01   25.5
1981-03-01   25.8
1981-04-01   26.7
1981-05-01   26.7
      REF_DATE   Population   Annual Change
361 1981-01-01     568000.0             4.8
362 1981-02-01     568000.0             4.8
363 1981-03-01     568000.0             4.8
364 1981-04-01     568000.0             4.8
365 1981-05-01     568000.0             4.8
    REF_DATE    VALUE
0 1981-01-01  17.0575
1 1981-02-01  17.1275
2 1981-03-01  16.9850
3 1981-04-01  17.0300
4 1981-05-01  18.5875
    REF_DATE  VALUE
0 1981-01-01  45000
1 1981-02-01  45000
2 1981-03-01  45000
3 1981-04-01  45000
4 1981-05-01  45000


In [283]:
# Merge housing data with interest rate data on REF_DATE
merged_data = housing_data.merge(interest_rate_monthly, on='REF_DATE', how='left', suffixes=('_housing', '_interest'))

# Merge with income data on REF_DATE
merged_data = merged_data.merge(income_monthly, on='REF_DATE', how='left')

# Merge with population data on REF_DATE
merged_data = merged_data.merge(population_monthly, on='REF_DATE', how='left')

# Merge with the previously merged dataset on REF_DATE
merged_data = merged_data.merge(median_house_price_data, on='REF_DATE', how='left')

# View the merged data
print(merged_data.head())


    REF_DATE  VALUE_housing  VALUE_interest    VALUE   Population  \
0 1981-01-01           25.3         17.0575  45000.0     568000.0   
1 1981-02-01           25.5         17.1275  45000.0     568000.0   
2 1981-03-01           25.8         16.9850  45000.0     568000.0   
3 1981-04-01           26.7         17.0300  45000.0     568000.0   
4 1981-05-01           26.7         18.5875  45000.0     568000.0   

    Annual Change  median_re_assessed_value  
0             4.8                       NaN  
1             4.8                       NaN  
2             4.8                       NaN  
3             4.8                       NaN  
4             4.8                       NaN  


In [284]:
# Renaming columns for more clarity
merged_data.rename(columns={
    'VALUE_housing': 'Housing_Price_Index',
    'VALUE_interest': 'Interest_Rate',
    'VALUE': 'Median_Income',
    'Population': 'Calgary_Population',
    'Annual Change': 'Annual_Population_Change',
    'median_re_assessed_value': 'Median_House_Price'
}, inplace=True)

# View the updated DataFrame
print(merged_data.tail())


    REF_DATE  Housing_Price_Index  Interest_Rate  Median_Income   Population  \
0 1981-01-01                 25.3        17.0575        45000.0     568000.0   
1 1981-02-01                 25.5        17.1275        45000.0     568000.0   
2 1981-03-01                 25.8        16.9850        45000.0     568000.0   
3 1981-04-01                 26.7        17.0300        45000.0     568000.0   
4 1981-05-01                 26.7        18.5875        45000.0     568000.0   

    Annual Change  Median_House_Price  
0             4.8                 NaN  
1             4.8                 NaN  
2             4.8                 NaN  
3             4.8                 NaN  
4             4.8                 NaN  


,REF_DATE,VALUE
488,2021-09-01,43400
489,2021-10-01,43400
490,2021-11-01,43400
491,2021-12-01,43400
492,2022-01-01,48000
